In [28]:
from PIL import Image
import numpy as np
from tqdm import tqdm

def read_image_to_array(file_path):
    with Image.open(file_path) as img:
        img = img.convert('RGB')

        image_array = np.array(img)

        return image_array

def rgb_to_ycbcr(img):
    # Coefficients for conversion from RGB to YCbCr
    xform = np.array([[0.299, 0.587, 0.114],
                      [-0.168736, -0.331264, 0.5],
                      [0.5, -0.418688, -0.081312]])
    
    ycbcr = img.dot(xform.T)
    ycbcr[:, :, [1, 2]] += 128   # Adding 128 to Cb and Cr channels
    return np.uint8(ycbcr)

def downsample_420(ycbcr_img):
    # Downsample the Cb and Cr channels by a factor of 2
    y = ycbcr_img[:, :, 0]
    cb = ycbcr_img[::2, ::2, 1]
    cr = ycbcr_img[::2, ::2, 2]

    return y, cb, cr

In [29]:
    
file_path = './ganyu.png'  
image_array = read_image_to_array(file_path)
ycbcr_image = rgb_to_ycbcr(image_array)
y, cb, cr = downsample_420(ycbcr_image)

print(cb.shape)


(256, 256)


In [30]:
def dct_2d(block):
    """Perform 2D Discrete Cosine Transform on an 8x8 block."""
    dct = np.zeros((8, 8))
    for u in range(8):
        for v in range(8):
            sum = 0
            for x in range(8):
                for y in range(8):
                    sum += block[x, y] * np.cos((2 * x + 1) * u * np.pi / 16) * np.cos((2 * y + 1) * v * np.pi / 16)
            sum *= 0.25 * (1 / np.sqrt(2) if u == 0 else 1) * (1 / np.sqrt(2) if v == 0 else 1)
            dct[u, v] = sum
    return dct

def apply_dct_to_image(img):
    """Apply DCT to each 8x8 block of the image."""
    h, w = img.shape
    dct_transformed = np.zeros((h, w))

    for i in tqdm(range(0, h, 8)):
        for j in range(0, w, 8):
            dct_transformed[i:i+8, j:j+8] = dct_2d(img[i:i+8, j:j+8])

    return dct_transformed

In [31]:
dct_y = apply_dct_to_image(y)
dct_cb = apply_dct_to_image(cb)
dct_cr = apply_dct_to_image(cr)

  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:08<00:00,  3.91it/s]


In [32]:
standard_quantization_matrix = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])
def quantize_block(dct_block, quant_matrix):
    return np.round(dct_block / quant_matrix).astype(int)
def apply_quantization(dct_image, quant_matrix):
    h, w = dct_image.shape
    quantized_image = np.zeros((h, w))

    for i in range(0, h, 8):
        for j in range(0, w, 8):
            quantized_image[i:i+8, j:j+8] = quantize_block(dct_image[i:i+8, j:j+8], quant_matrix)

    return quantized_image


In [33]:
quantized_y = apply_quantization(dct_y, standard_quantization_matrix)
quantized_cb = apply_quantization(dct_cb, standard_quantization_matrix)
quantized_cr = apply_quantization(dct_cr, standard_quantization_matrix)

In [34]:
def zigzag(input):
    h, w = input.shape
    result = np.zeros(h * w)
    index = -1

    for i in range(h + w - 1):
        if i % 2 == 0:
            x = i if i < h else h - 1
            y = 0 if i < h else i - h + 1
            while x >= 0 and y < w:
                index += 1
                result[index] = input[x, y]
                x -= 1
                y += 1
        else:
            x = 0 if i < w else i - w + 1
            y = i if i < w else w - 1
            while x < h and y >= 0:
                index += 1
                result[index] = input[x, y]
                x += 1
                y -= 1

    return result

def run_length_encoding(input):
    symbols = []
    count = 0
    for elem in input:
        if elem == 0:
            count += 1
        else:
            symbols.append((count, elem))
            count = 0
    if count > 0:
        symbols.append((0, 0))
    return symbols

In [35]:
runlen_y = run_length_encoding(zigzag(quantized_y))
runlen_cb = run_length_encoding(zigzag(quantized_cb))
runlen_cr = run_length_encoding(zigzag(quantized_cr))

In [36]:
class Node:
    def __init__(self, symbol, frequency):
        self.symbol = symbol
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency

        
import heapq
from bitarray import bitarray

def build_huffman_tree(symbols_frequency):
    heap = [Node(symbol, frequency) for symbol, frequency in symbols_frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = Node(None, left.frequency + right.frequency)
        merged.left = left
        merged.right = right

        heapq.heappush(heap, merged)

    return heap[0]

def generate_huffman_codes(node, prefix=bitarray(), code={}):
    if node is not None:
        if node.symbol is not None:
            code[node.symbol] = prefix.copy()
        generate_huffman_codes(node.left, prefix + bitarray('0'), code)
        generate_huffman_codes(node.right, prefix + bitarray('1'), code)
    return code

def encode_huffman(data, huffman_codes):
    merge_bitarray = bitarray()
    for symbol in data:
        merge_bitarray.extend(huffman_codes[symbol])
    return merge_bitarray

def compute_frequency(data):
    frequency = {}
    for symbol in data:
        if symbol not in frequency:
            frequency[symbol] = 1
        else:
            frequency[symbol] += 1
    return frequency


In [40]:
# perform the huffman encoding
freq_y = compute_frequency(runlen_y)
huffman_tree_y = build_huffman_tree(freq_y)
huffman_code_y = generate_huffman_codes(huffman_tree_y, bitarray())
data_y = encode_huffman(runlen_y, huffman_code_y)

freq_cb = compute_frequency(runlen_cb)
huffman_tree_cb = build_huffman_tree(freq_cb)
huffman_code_cb = generate_huffman_codes(huffman_tree_cb, bitarray())
data_cb = encode_huffman(runlen_cb, huffman_code_cb)

freq_cr = compute_frequency(runlen_cr)
huffman_tree_cr = build_huffman_tree(freq_cr)
huffman_code_cr = generate_huffman_codes(huffman_tree_cr, bitarray())
data_cr = encode_huffman(runlen_cr, huffman_code_cr)



In [45]:
print(data_cb[:10])
print(huffman_code_cb[runlen_cb[0]])
for symbol, code in huffman_code_y.items():
    print(f'symbol={symbol} code={code}')

bitarray('1100000100')
bitarray('110000010001')
symbol=(0, 1.0) code=bitarray('1110')
symbol=(0, -3.0) code=bitarray('1101000')
symbol=(7, 26.0) code=bitarray('001010000')
symbol=(5, -5.0) code=bitarray('0010100010')
symbol=(6, 11.0) code=bitarray('00101000110')
symbol=(6, 13.0) code=bitarray('011100010110')
symbol=(7, 27.0) code=bitarray('001010010')
symbol=(7, 7.0) code=bitarray('01110011011')
symbol=(0, 18.0) code=bitarray('00101001110')
symbol=(7, 106.0) code=bitarray('00101001111')
symbol=(3, 1.0) code=bitarray('0110110100')
symbol=(5, -7.0) code=bitarray('00101011000')
symbol=(28, 1.0) code=bitarray('01101111101')
symbol=(24, -1.0) code=bitarray('1111110001')
symbol=(7, 36.0) code=bitarray('001010111')
symbol=(0, -12.0) code=bitarray('0010110000')
symbol=(7, 13.0) code=bitarray('0010110001')
symbol=(7, 81.0) code=bitarray('0100010')
symbol=(7, 67.0) code=bitarray('100111')
symbol=(7, 70.0) code=bitarray('0000010')
symbol=(7, 84.0) code=bitarray('111111111')
symbol=(7, 90.0) code=

In [46]:
      
def decode_huffman(encoded_data, root):
    decoded_output = []
    current_node = root
    for bit in encoded_data:
        if bit == 0:
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.left is None and current_node.right is None:  # Leaf node
            decoded_output.append(current_node.symbol)
            current_node = root  # Reset for next symbol

    return decoded_output

def rebuild_huffman_tree(huffman_codes):
    root = Node(None, 0)
    for symbol, code in huffman_codes.items():
        current_node = root
        for bit in code:
            if bit == 0:
                if current_node.left is None:
                    current_node.left = Node(None, 0)
                current_node = current_node.left
            else:
                if current_node.right is None:
                    current_node.right = Node(None, 0)
                current_node = current_node.right
        current_node.symbol = symbol
    return root

# Decode Huffman
decode_huffman_tree_y = rebuild_huffman_tree(huffman_code_y)
decode_huffman_tree_cb = rebuild_huffman_tree(huffman_code_cb)
decode_huffman_tree_cr = rebuild_huffman_tree(huffman_code_cr)

decode_y = decode_huffman(data_y, huffman_tree_y)


AttributeError: 'NoneType' object has no attribute 'left'

In [ ]:
import pickle

my_dict = {'huffman_code_y': huffman_code_y, 'y': data_y, 'huffman_code_cb': huffman_code_cb, 
           'cb': data_cb, 'huffman_code_cr': huffman_code_cr, 'cr': data_cr}

# # Write to file
with open('output.pkl', 'wb') as f:
    pickle.dump(my_dict, f)